In [1]:
import xarray as xr
from pathlib import Path
from CMIP6_IO import CMIP6_IO
import os

## Create ensemble files from running CMIP6_light.py
This script will loop over all files found in a speficic folder `lightpath`and find files of teh same variable and scenario to create ensemble files from. The various variables to create ensembles for include `["uvb_","uv_","uvi_","par_", "uvb_", "uva_"]`.

The output is stored under folder `ensemble_path` and the results are used to create the final plots of modeled lightlyfor the paper using notebooks `CMIP6_plot_light_results.ipynb`.


In [2]:
# Define what models, scenarios, and variables to extract for weight creation

source_ids = ["CMCC-ESM2"] #, "CanESM5", "MPI-ESM1-2-LR", "UKESM1-0-LL", "MPI-ESM1-2-HR"]
scenarios = ["ssp245"]
var_names = ["tos"]
lightpath = "light"
io = CMIP6_IO()

In [3]:
for var_name in var_names:
    for scenario in scenarios:
        for source_id in source_ids:
            weights_path = f"{lightpath}/weights/{var_name}"
            if not os.path.exists(weights_path):
                os.makedirs(weights_path, exist_ok=True)
            current = f"{lightpath}/{scenario}/{source_id}"
            file_on_gcs = io.list_dataset_on_gs(current)
           
            min_lon = 5
            max_lon = 35    
            min_lat = 69
            max_lat = 79
            for f in file_on_gcs:
                # Filter to only use the models we prefer (UKESM1-0-LL excluded)
                if any(source_id in f for source_id in source_ids) and var_name in f and "weights" not in f:
                    ds = io.open_dataset_on_gs(f)
                    print(ds)
                   # ds=(ds.assign_coords(x=(((ds.x + 180) % 360) - 180))).sortby('x')
                    ds=ds.assign_coords(lat=ds.lat[:,0].values, lon=ds.lon[0,:].values)
                  #  ds=(ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180))).sortby('lon') 
                    ds_clipped = ds.sel(lat=slice(min_lat,max_lat), lon=slice(min_lon, max_lon)) 
                    print(f"Opening {var_name} - {f} {ds}")
                   
                    # We cut out a specific region approximately equal to the Barents Sea LME for weight calculations.
                    
                    
                    # Store the cut oiut region as new file and upload to GCS
                    filename = f"{weights_path}/{f.split('/')[-1][0:-3]}_clipped.nc"
                    ds_clipped.to_netcdf(filename)
            
                    io.upload_to_gcs(filename)

                    # Delete the local version of the file
                    if Path(filename).exists():
                        Path(filename).unlink()
            
                    print(f"Created clipped file {filename}")
                    print(ds_clipped)
                    ds.close()

[CMIP6_IO] Opening file actea-shared/light/ssp245/CMCC-ESM2/CMIP6_CMCC-ESM2_r1i1p1f1_ssp245_tos.nc
Opening tos - light/ssp245/CMCC-ESM2/CMIP6_CMCC-ESM2_r1i1p1f1_ssp245_tos.nc <xarray.Dataset>
Dimensions:      (time: 1284, y: 50, x: 720, bnds: 2, lon: 720, lat: 50)
Coordinates:
  * time         (time) object 1993-01-16 00:00:00 ... 2099-12-16 00:00:00
    time_bounds  (time, bnds) object ...
  * lon          (lon) float64 0.5 1.0 1.5 2.0 2.5 ... 358.5 359.0 359.5 360.0
  * lat          (lat) float64 60.5 61.0 61.5 62.0 62.5 ... 83.5 84.0 84.5 85.0
Dimensions without coordinates: y, x, bnds
Data variables:
    tos          (time, y, x) float32 ...
    mask         (y, x) int64 ...
Attributes:
    regrid_method:  bilinear
Created clipped file light/weights/tos/CMIP6_CMCC-ESM2_r1i1p1f1_ssp245_tos_clipped.nc
<xarray.Dataset>
Dimensions:      (time: 1284, y: 50, x: 720, bnds: 2, lon: 61, lat: 21)
Coordinates:
  * time         (time) object 1993-01-16 00:00:00 ... 2099-12-16 00:00:00
    time